In [83]:
import torch
import pandas as pd
import os
import sys
import numpy as np
import gym
from collections import defaultdict

In [84]:
dir_path = os.path.dirname(os.path.realpath('__file__'))
parent_dir = f"../src"
sys.path.insert(0, parent_dir) 

from QNetwork import QNetwork

In [85]:
env_name = "MountainCar-v0"
env = gym.envs.make(env_name)

dir = f"{dir_path}/../data/{env_name}"

model = torch.load(f"{dir}/weights.pt")
d = torch.load(f"{dir}/results.pkl")
trajectories = torch.load(f"{dir}/trajectories.pkl")

discount = 0.9

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [91]:
trajectory_dict = defaultdict(dict)
for i, trajectory_tuple in enumerate(trajectories):
    trajectory, seed = trajectory_tuple
    trajectory_dict[i]["seed"] = seed
    trajectory_dict[i]["trajectory"] = trajectory
    trajectory_dict[i]["episode_length"] = len(trajectory)
    
    # Not really a way to account for the agent finishing right in the last second
    trajectory_dict[i]["is_finished"] = len(trajectory) < env._max_episode_steps
    _, _, rewards, _, _ = zip(*trajectory)
    
    trajectory_dict[i]["max_reward"] = np.max(rewards)
    trajectory_dict[i]["sum_reward"] = np.sum(rewards)
    discounts = [np.power(discount, i) for i in range(len(rewards))][::-1]
    trajectory_dict[i]["sum_discounted_reward"] = np.sum([rewards[i] * discounts[i] for i in range(len(rewards))])

In [92]:
df = pd.DataFrame.from_dict(trajectory_dict).T

In [93]:
df.head()

,episode_length,is_finished,max_reward,seed,sum_discounted_reward,sum_reward,trajectory
0,200,False,-1,0,-10,-200,"[([-0.5891279887498433, 0.0], 2, -1.0, [-0.587..."
1,200,False,-1,1,-10,-200,"[([-0.4385219128542218, 0.0], 2, -1.0, [-0.438..."
2,200,False,-1,2,-10,-200,"[([-0.43062342592296066, 0.0], 0, -1.0, [-0.43..."
3,200,False,-1,3,-10,-200,"[([-0.45437538945067857, 0.0], 0, -1.0, [-0.45..."
4,200,False,-1,4,-10,-200,"[([-0.4491204486681113, 0.0], 1, -1.0, [-0.449..."
